In [21]:
# DICE - Model

# IMPORT PACKAGES & SET PATH
import numpy as np
import pandas as pd
from scipy.stats import norm, skewnorm, cauchy, lognorm
import logging

import json
import sys
import os
# pydice_folder = os.path.dirname(os.getcwd())+"\\1_Model"
pydice_folder = os.path.dirname(os.getcwd()+"\\06_Code")
sys.path.insert(1, pydice_folder)


In [22]:
    def __init__(self, tstep=5, steps=60, model_specification="EMA"):
        self.tstep = tstep					# (in years)
        self.steps = steps
        self.startYear = 2010
        self.tperiod = []
        self.model_specification = model_specification
        for i in range(0, self.steps):
            self.tperiod.append((i*self.tstep)+self.startYear)


        with open(pydice_folder + '\\ecs_dist_v4.json') as f:
            d=json.load(f)
        
        #creating a list from the dist of t2xC02
        np.random.seed(10)

        minb = 0
        maxb = 20
        nsamples = 1000

        samples_norm = np.zeros((0,))
        while samples_norm.shape[0] < nsamples:
            samples = (norm.rvs(d['norm'][0],d['norm'][1],nsamples))
            accepted = samples[(samples >= minb) & (samples <= maxb)]
            samples_norm = np.concatenate((samples_norm, accepted), axis=0)
        samples_norm = samples_norm[:nsamples]

        samples_lognorm = np.zeros((0,))
        while samples_lognorm.shape[0] < nsamples:
            samples = (lognorm.rvs(d['lognorm'][0],d['lognorm'][1],d['lognorm'][2],nsamples))
            accepted = samples[(samples >= minb) & (samples <= maxb)]
            samples_lognorm = np.concatenate((samples_lognorm, accepted), axis=0)
        samples_lognorm = samples_lognorm[:nsamples]

        samples_cauchy = np.zeros((0,))
        while samples_cauchy.shape[0] < nsamples:
            samples = (cauchy.rvs(d['cauchy'][0],d['cauchy'][1],nsamples))
            accepted = samples[(samples >= minb) & (samples <= maxb)]
            samples_cauchy = np.concatenate((samples_cauchy, accepted), axis=0)
        samples_cauchy = samples_cauchy[:nsamples]

        # extend array with the deterministic value of the nordhaus
        samples_norm = np.append(samples_norm, 2.9)
        samples_lognorm = np.append(samples_lognorm, 2.9)
        samples_cauchy = np.append(samples_cauchy, 2.9)
        
        self.samples_t2xco2 = [samples_norm, samples_lognorm, samples_cauchy]


In [23]:
   def __call__(self,
                 # uncertainties from Nordhaus(2008) (first draft)
                 t2xco2_index=-1,
                 t2xco2_dist=0,
                 tfp_gr=0.079,
                 sigma_gr=-0.01, # growth rate
                 pop_gr=0.134,
                 fosslim=6000,
                 cback=344,
                 decl_back_gr=0.025,
                 limmiu=1.2,
                 fdamage=0,
                 # levers from Nordhaus(2008) (first draft)
                 sr=0.249,#shridhar: savings rate fixed
                 prtp_con=0.015, #shridhar: prtp consumption
                 prtp_dam=0, #shridhar: prtp damage, 
                 periodfullpart=21,
                 miu_period=29, #17
                 **kwargs):

        """
        ####################################################################
        ######################### MODEL-SET UPS ############################
        ####################################################################
        """
        """
        ####################### ECONOMICS PARAMETERS #######################
        """
        self.pop = np.zeros((self.steps,))
        self.tfp = np.zeros((self.steps,))
        self.tfp_gr = np.zeros((self.steps,))
        self.k = np.zeros((self.steps,))
        self.i = np.zeros((self.steps,))
        self.ygross = np.zeros((self.steps,))
        self.ynet = np.zeros((self.steps,))     # ynet = net output damages = ygross * damfrac
        self.damfrac = np.zeros((self.steps,))

# income = production - damages (as eq to demand drop)
# rename ynet as income

        self.damages = np.zeros((self.steps,))  # damages = ygross* damfrac
        self.pbacktime = np.zeros((self.steps,))
        self.cost1 = np.zeros((self.steps,)) #rename to self.backstop_cost
        self.partfract = np.zeros((self.steps,))
        self.abatecost = np.zeros((self.steps,))
        # self.miu_up = np.zeros((self.steps,))
        self.sigma = np.zeros((self.steps,))
        self.sigma_gr = np.zeros((self.steps,))
        self.eind = np.zeros((self.steps,))
        self.cumetree = np.zeros((self.steps,))
        self.etree = np.zeros((self.steps,))
        self.e = np.zeros((self.steps,)) #total emission
        self.cca = np.zeros((self.steps,)) #cumulative emissions (rename)
        self.y = np.zeros((self.steps,)) #this is actually income. it is y-damages-abatementcosts. 
        self.c = np.zeros((self.steps,))
        self.cpc = np.zeros((self.steps,)) #look at formula in init and model, why is there a /1000?
        
        self.con_g = np.zeros((steps,)) # consumption growth, iitializaed to 0
        self.dam_g = np.zeros((steps,)) # damage growth, iitializaed to 0

        self.sdr_con = np.zeros((self.steps,))
        self.sdr_dam = np.zeros((self.steps,))

        self.consumption_sdf = np.zeros((self.steps,))
        self.damage_sdf = np.zeros((self.steps,))

        self.inst_util_con = np.zeros((self.steps,))
        self.disc_util_con = np.zeros((self.steps,))
        
        self.inst_disutil_dam = np.zeros((self.steps,))
        self.disc_disutil_dam = np.zeros((self.steps,))
        
        self.welfare = np.zeros((self.steps,))
        
        self.cprice = np.zeros((self.steps,))
        self.ccatot = np.zeros((self.steps,))
        self.scc = np.zeros((self.steps,))
        self.atfrac = np.zeros((self.steps,))
        self.atfrac2010 = np.zeros((self.steps,))
        self.ppm = np.zeros((self.steps,))

        """
        ######################### CARBON PARAMETERS ########################
        """

        self.mat = np.zeros((self.steps,))
        self.mu = np.zeros((self.steps,))
        self.ml = np.zeros((self.steps,))
        self.forcoth = np.zeros((self.steps,))
        self.forc = np.zeros((self.steps,))

        """
        ######################## CLIMATE PARAMETERS ########################
        """

        # Increase temperature of atmosphere [dC from 1900]
        self.temp_atm = np.zeros((self.steps,))
        # Increase temperature of lower oceans [dC from 1900]
        self.temp_ocean = np.zeros((self.steps,))

        """
        ########################## DEEP UNCERTAINTIES ######################
        """

        # Equilibrium temperature impact [dC per doubling CO2]/
        # CLimate sensitivity parameter (2.9)
        self.t2xco2 = self.samples_t2xco2[t2xco2_dist][t2xco2_index]
        
        # Choice of the damage function (structural deep uncertainty)
        self.fdamage = fdamage


        """
        ############################# LEVERS ###############################
        """
        if self.model_specification == 'EMA':
            self.miu = np.zeros((self.steps,))
        else:
            DICE_OPT = pd.read_excel("DICE2013R.xlsm", sheet_name="Opttax",
                                     index_col=0)
            self.miu = np.array(DICE_OPT.iloc[133])
# shridhar: self.s defined here = savings rate, or it is actually just savings values, even though it is called self.sr. look at initialization of econ parameters, investment and savings: self.s[0] = self.sr
        if self.model_specification == 'EMA':
            self.s = np.zeros((self.steps,))
        else:
            self.s = np.array(DICE_OPT.iloc[129]) #shridhar: # todo what does this do?

        # Initial Consumption discount rate (per year) (0.015)
        # self.irstp = irstp
        self.sdr_con[t] = discount_dam

        # Initial rate of social time preference (per year) (1)
        self.sdr_dam[t] = discount_dam  #todo this needs to be updated


        # Upper limit on control rate after 2150
        self.limmiu = limmiu
        # Initial emissions control rate for base case 2015
        self.miu0 = 0.03
        if self.model_specification == 'EMA':
            self.miu[0] = self.miu0
        self.miu_period = miu_period
        # Savings rate (optlrsav = 0.2582781457) from the control file
    #shridhar: savings rate is specified here, this determines investments & therefore savings
        self.sr = sr


In [24]:
   def __call__(self,
                 # uncertainties from Nordhaus(2008) (first draft)
                 t2xco2_index=-1,
                 t2xco2_dist=0,
                 tfp_gr=0.079,
                 sigma_gr=-0.01, # growth rate
                 pop_gr=0.134,
                 fosslim=6000,
                 cback=344,
                 decl_back_gr=0.025,
                 limmiu=1.2,
                 fdamage=0,
                 # levers from Nordhaus(2008) (first draft)
                 sr=0.249,#shridhar: savings rate fixed
                 prtp_con=0.015, #shridhar: prtp consumption
                 prtp_dam=0, #shridhar: prtp damage, 
                 periodfullpart=21,
                 miu_period=29, #17
                 **kwargs):
        """
        ################# ECONOMIC PARAMETER INTITIALISATION ###############
        """

        # Initial world population (in Millions [2015])
        self.pop[0] = 6838.0

        # Initial level of total factor productivity (TFP)
        self.tfp[0] = 3.80
        # Initial growth rate for TFP (per 5 years)
        self.tfp_gr[0] = 0.079

        # Initial capital value 2015 [Trillions 2010 US $]
        self.k[0] = 135.0

        # Gross world product: gross abatement and damages
        # Gama: Capital elasticity in production function
        self.ygross[0] = (self.tfp[0]*((self.pop[0]/1000)**(1-self.gama))
                          * (self.k[0]**self.gama))

        # Damage Fraction/temp: Temperature/a1: Damage intercept
        # shridhar: damage function chosen here
        # a2: Damage quadratic term/a3: Damage exponent        
        # self.damfrac[0] = (self.a1*self.temp_atm[0]
        #                   + self.a2*(self.temp_atm[0]**self.a3))
        if self.fdamage == 0:
            self.damfrac[0] = (self.a1*self.temp_atm[0] 
                               + self.a2*(self.temp_atm[0]**self.a3))
        elif self.fdamage == 1:
            self.damfrac[0] = (1-(np.exp(-0.0025*self.temp_atm[0]**2.45)))
            
        elif self.fdamage == 2:
            self.damfrac[0] = (1-1/(1+0.0028388**2+0.0000050703
                                    *(self.temp_atm[0]**6.754)))
        # Net output damages
        self.ynet[0] = self.ygross[0]*(1.0-self.damfrac[0])
        # Damages
        self.damages[0] = self.ygross[0]*self.damfrac[0]

        # Initial damage growth rate
        self.dam_g[0] = 0.00

        # Output-to-Emission
        # Industrial emissions 2010 [GtCO2 per year]
        self.e0 = 33.61
        # Initial world gross output [Trillions 2015 US $]
        self.q0 = 63.69
        # CO2-equivalent-emissions to output ratio
        self.sigma[0] = self.e0/(self.q0 * (1 - self.miu[0]))
        # Change in sigma: the cumulative improvement in energy efficiency)
        # Initial growth of sigma (per year)
        self.sigma_gr[0] = sigma_gr

        # Backstop price/cback: cost of backstop
        # decl_back_gr: decline of backstop
        self.pbacktime[0] = self.cback
        # Adjusted cost for backstop
        self.cost1[0] = self.pbacktime[0]*self.sigma[0]/self.expcost2/1000

        # Fraction of emissions under control regime
        if self.periodfullpart == 0:
            self.partfract[0] = 1
        else:
            self.partfract[0] = self.partfract2010

        # Abatement costs
        self.abatecost[0] = (self.ygross[0]*self.cost1[0]
                             * (self.miu[0]**self.expcost2)
                             * (self.partfract[0]**(1-self.expcost2)))

        # Carbon price
        self.cprice[0] = (self.pbacktime[0]
                          * ((self.miu[0]/self.partfract[0])
                             ** (self.expcost2-1)))

        # Industrial emissions
        self.eind[0] = self.sigma[0]*self.ygross[0]*(1.0-self.miu[0])

        # Emissions from deforestation
        self.etree[0] = self.eland0
        # Cumlative emission from land/tree
        self.cumetree[0] = 100

        # Total emissions
        self.e[0] = self.eind[0]+self.etree[0]

        # Cumulative emissions (cca_up - fossil limits)
        self.cca[0] = 90.0
        # if (self.cca[0] > self.cca_up): #shridhar: how will this (initial) value be < 90 when it's just been defined
        #     self.cca[0] = self.cca_up + 1.0

        self.ccatot[0] = self.cca[0] + self.etree[0]

        # Gross world product (net of abatement and damages)
        self.y[0] = self.ynet[0]-self.abatecost[0]
        #askshajee: why is there a limit to output without affecting any production inputs? I'm taking this out to capture actual growth effects
        # if self.y[0] < self.y_lo: #y_lo defined as zero.
        #     self.y[0] = self.y_lo 

        # Investments & Savings
        if self.model_specification == 'EMA':
            self.s[0] = self.sr
        self.i[0] = self.s[0]*self.y[0]

        # Consumption
        # yearly consumption
        self.c[0] = self.y[0] - self.i[0]
        # consumption per capita
        self.cpc[0] = self.c[0]*1000/self.pop[0]

        #Consumption growth rate
        self.con_g[0] = 0.00 

        # Utility of consumption
        # consumption utility social discount rate
        
        self.sdr_con[0] = (prtp_con + (self.emuc * self.con_g[0]))
        self.consumption_sdf[0] = 1.00/((1.00 + self.sdr_con[0])**self.tstep)

        # Instantaneous utility 
        #Shridhar: I'm changing this from U(C)= C^((1-emuc)-1)/((1-emuc)-1). check func visualization, this is wrong
        self.inst_util_con[0] = (((self.cpc[0])
                              ** (1.0-self.emuc))
                             / (1.0-self.emuc))

        # discounted utility of consumption
        self.disc_util_con[0] = self.inst_util_con[0] * self.pop[0] * self.consumption_sdf[0]

        #Disutility of damage
        # damage disutility social discount rate and factor
        
        self.sdr_dam[0] = prtp_dam + (self.emdd * self.damages[0])
        self.damage_sdf[0] = 1.00 /((1.00 + self.sdr_con[0])**self.tstep)

        # Instantaneous disutility 
        self.inst_disutil_dam[0] = (((self.damages[0]) ** (1.0 - self.emdd))/ (1.0 - self.emdd))

        # Discounted disutility of damage
        self.disc_disutil_dam[0] = self.inst_disutil_dam[0] * self.pop[0] * self.damage_sdf[0]

        # Welfare function
        # shridhar: check the whole scaling thing and verify formula
        self.welfare[0] = ((self.tstep*self.scale1*np.sum(self.disc_util_con - self.disc_disutil_dam))
                        + self.scale2)
        
       
        # logging.info(self, "is initialized.")


In [39]:
   def __call__(self,
                 # uncertainties from Nordhaus(2008) (first draft)
                 t2xco2_index=-1,
                 t2xco2_dist=0,
                 tfp_gr=0.079,
                 sigma_gr=-0.01, # growth rate
                 pop_gr=0.134,
                 fosslim=6000,
                 cback=344,
                 decl_back_gr=0.025,
                 limmiu=1.2,
                 fdamage=0,
                 # levers from Nordhaus(2008) (first draft)
                 sr=0.249,#shridhar: savings rate fixed
                 prtp_con=0.015, #shridhar: prtp consumption
                 prtp_dam=0, #shridhar: prtp damage, 
                 periodfullpart=21,
                 miu_period=29, #17
                 **kwargs):


    for t in range(1, self.steps):

                """
                ####################### CARBON SUB-MODEL #######################
                """

# Carbon concentration increase in atmosphere [GtC from 1750]
self.mat[t] = ((self.e[t-1]*(5.0/3.666)) + self.b11*self.mat[t-1]+self.b21*self.mu[t-1])
if(self.mat[t] < self.mat_lo):
    self.mat[t] = self.mat_lo
    
# Carbon concentration increase in shallow oceans [GtC from 1750]
self.mu[t] = (self.b12*self.mat[t-1]+self.b22*self.mu[t-1]
                          + self.b32*self.ml[t-1])
if(self.mu[t] < self.mu_lo):
    self.mu[t] = self.mu_lo

# Carbon concentration increase in lower oceans [GtC from 1750]
self.ml[t] = self.b33*self.ml[t-1]+self.b23*self.mu[t-1]
if(self.ml[t] < self.ml_lo):
    self.ml[t] = self.ml_lo

# Radiative forcing
# Exogenous forcings from other GHG
if (t < 19):
    self.forcoth[t] = self.fex0+(1.0/18.0)*(self.fex1-self.fex0)*t
else:
    self.forcoth[t] = self.fex0+(self.fex1-self.fex0)


NameError: name 'self' is not defined